In [1]:
%%sh
## Dependencies
pip install pydotplus
pip install tqdm

  Created wheel for pydotplus: filename=pydotplus-2.0.2-cp36-none-any.whl size=23674 sha256=7dfb5df7b5dbc090124ddbcd72a444a1b92428268e1d1dd2fa009ad159a5b787
  Stored in directory: /home/ec2-user/.cache/pip/wheels/35/7b/ab/66fb7b2ac1f6df87475b09dc48e707b6e0de80a6d8444e3628
Successfully built pydotplus


You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
%load_ext autoreload
%autoreload 2

# helper python file
import ml_pipeline as pipeline
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [4]:
(train_df,test_df)=pipeline.read_input_data()
(seq_train,seq_test,word_index)=pipeline.tokenize_data(train_df,test_df)

  2%|▏         | 3419/159571 [00:00<00:04, 34187.02it/s]

num train:  159571
num test:  153164
pre-processing train data...


100%|██████████| 153164/153164 [00:03<00:00, 38763.95it/s]


tokenizing input data...
dictionary size:  282101


In [5]:
label_names = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [6]:
embedding_matrix=pipeline.prepare_embeddings("Glove-Twitter",word_index)

1193514it [09:06, 2184.10it/s]


preparing embedding matrix...
number of null word embeddings: 2316


In [7]:
model_name='bilstm_cnn_glove_twitter_cv'

In [11]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D
from keras.layers import Dropout, Embedding
from keras.preprocessing import text, sequence
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
from keras.models import Model

def init_BiLSTM_Conv_Model(embedding_matrix):
    # Build Model
    inp = Input(shape=(max_seq_len,))

    x = Embedding(max_features, embedding_matrix.shape[1], weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.35)(x)

    x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))(x)
    x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)

    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])

    out = Dense(6, activation='sigmoid')(x)

    model = Model(inp, out)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [13]:
max_features=30000
max_seq_len=150

In [ ]:
#5-Fold Cross-Validation

kf = KFold(n_splits=5)
nfold=1
cvscores = []

#training params
batch_size = 128 
num_epochs = 5 

for train_index, test_index in kf.split(seq_train):
    print(f' In Fold {nfold}... : \n')
    print("TRAIN:", len(train_index), "TEST:", len(test_index))
    
    seq_train_split=seq_train[train_index]
    y_train_split = train_df[label_names].iloc[train_index ].values
    
    # train the model
    model=init_BiLSTM_Conv_Model(embedding_matrix)
    hist=pipeline.train_model(model,num_epochs,batch_size,seq_train_split,y_train_split)
    
    # evaluate the model
    seq_test_split=seq_train[test_index]
    y_test_split = train_df[label_names].iloc[test_index ].values
    scores = model.evaluate(seq_test_split, y_test_split, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    
    # save the model
    pipeline.save_model(model, f'models/{model_name}-{nfold}')
    cvscores.append(scores[1] * 100)
    
    # save OOF predictions
    y_test = model.predict(seq_test_split)
    pipeline.create_submission_file(train_df.iloc[test_index],y_test,f'outputs/{model_name}_{nfold}_oof.csv')
    
    nfold +=1

 In Fold 1... : 

TRAIN: 127656 TEST: 31915










Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 150, 200)     6000000     input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 150, 200)     0           embedding_1[0][0]                
__________________________________________

Train on 102125 samples, validate on 25532 samples
Epoch 1/5
102125/102125 [==============================] - 429s 4ms/step - loss: 0.0663 - acc: 0.9764 - val_loss: 0.0486 - val_acc: 0.9821
Epoch 2/5
102125/102125 [==============================] - 423s 4ms/step - loss: 0.0486 - acc: 0.9818 - val_loss: 0.0484 - val_acc: 0.9815
Epoch 3/5
102125/102125 [==============================] - 424s 4ms/step - loss: 0.0458 - acc: 0.9824 - val_loss: 0.0447 - val_acc: 0.9828
Epoch 4/5
102125/102125 [==============================] - 423s 4ms/step - loss: 0.0438 - acc: 0.9830 - val_loss: 0.0443 - val_acc: 0.9833
Epoch 5/5
102125/102125 [==============================] - 421s 4ms/step - loss: 0.0422 - acc: 0.9836 - val_loss: 0.0441 - val_acc: 0.9832
acc: 98.30%
Saved model to disk
(31914, 7)
 In Fold 3... : 

TRAIN: 127657 TEST: 31914
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Con

Train on 102125 samples, validate on 25532 samples
Epoch 1/5
102125/102125 [==============================] - 433s 4ms/step - loss: 0.0652 - acc: 0.9774 - val_loss: 0.0459 - val_acc: 0.9832
Epoch 2/5
102125/102125 [==============================] - 428s 4ms/step - loss: 0.0486 - acc: 0.9816 - val_loss: 0.0436 - val_acc: 0.9839
Epoch 3/5
102125/102125 [==============================] - 423s 4ms/step - loss: 0.0461 - acc: 0.9823 - val_loss: 0.0434 - val_acc: 0.9835
Epoch 4/5
102125/102125 [==============================] - 419s 4ms/step - loss: 0.0436 - acc: 0.9831 - val_loss: 0.0417 - val_acc: 0.9845
Epoch 5/5
102016/102125 [============================>.] - ETA: 0s - loss: 0.0421 - acc: 0.9836

In [15]:
cvscores

[98.2986042997446,
 98.30116231974753,
 98.35599707490319,
 98.3246627715758,
 98.36487507412231]

In [17]:
best_model_index=cvscores.index(max(cvscores))+1
print (f'best_model_index : {best_model_index}')
best_model=pipeline.load_model(f'models/{model_name}-{best_model_index}')

best_model_index : 5
Loaded model from disk


In [18]:
y_test = best_model.predict(seq_test)
pipeline.create_submission_file(test_df,y_test,f'outputs/{model_name}_submission.csv')

(153164, 7)


In [19]:
# concatenate OOF files
dataframes = []
filenames=[1,2,3,4,5]
for i in filenames:
    dataframes.append(pd.read_csv(f'outputs/{model_name}_{i}_oof.csv'))
oof=dataframes[0]
for df in dataframes[1:]:
    oof=oof.append(df)
print (oof.shape)
oof.to_csv(f'outputs/{model_name}_oof.csv', index=False)

(159571, 7)
